## Example for computing parallel imaging g-factor using BART Python interface

Author: Jon Tamir <jtamir@eecs.berkeley.edu>

Data is taken from the ISMRM 2013 Sunrise Course on Parallel Imaging  
http://hansenms.github.io/sunrise/sunrise2013/

For more information, see the following papers:

Kellman, P. and McVeigh, E. R. (2005), Image reconstruction in SNR units: A general method for SNR measurement. Magn. Reson. Med., 54: 1439-1447. doi:10.1002/mrm.20713

Robson, P. M., Grant, A. K., Madhuranthakam, A. J., Lattanzi, R. , Sodickson, D. K. and McKenzie, C. A. (2008), Comprehensive quantification of signal‐to‐noise ratio and g‐factor for image‐based and k‐space‐based parallel imaging reconstructions. Magn. Reson. Med., 60: 895-907. doi:10.1002/mrm.21728


### Requirements
- Install bart (version 0.4.02 or newer): https://github.com/mrirecon/bart  
- Add bart to PATH and PYTHONPATH in .bashrc:
```bash
export PATH=/path/to/bart:$PATH  
export PYTHONPATH=/path/to/bart/python:$PYTHONPATH
```
- Install python dependencies:  
`pip install h5py numpy matplotlib`

In [ ]:
import numpy as np
import h5py
import time

import cfl
from bart import bart

import matplotlib.pyplot as plt
%matplotlib inline

from pylab import rcParams
rcParams['figure.figsize'] = 7, 7

## Load data from hdf5 file
The data is taken from http://hansenms.github.io/sunrise/sunrise2013/ and saved as an hdf5 file. The data are Cartesian k-space prospectively under-sampled with a 1x4 acceleration factor. The sensitivity maps were computed from a pre-scan

In [ ]:
print('loading data')

with h5py.File('data.h5', 'r') as F:
    ksp = np.array(F['ksp'])
    sens = np.array(F['sens'])
    mask = np.array(F['mask'])
    noise = np.array(F['noise'])
    
print('done!')
print('ksp shape:', ksp.shape)
nx, ny, nc = ksp.shape
cimg = bart(1, 'fft -iu 3', ksp)

plt.figure(figsize=(16,20))
for i in range(nc):
    plt.subplot(1, nc, i+1)
    plt.imshow(abs(cimg[:,:,i]).squeeze(), cmap='gray')
    plt.title('Coil image {}'.format(i))
    

plt.figure(figsize=(16,20))
for i in range(nc):
    plt.subplot(1, nc, i+1)
    plt.imshow(abs(sens[:,:,i]).squeeze(), cmap='gray')
    plt.title('Sensitivity map {}'.format(i))


## Noise whitening
Notice that some coil images have higher noise than others. We can estimate the noise covariance matrix from our noise measurements and apply a noise pre-whitening matrix. We need to apply it to both the data and the sensitivity maps

In [ ]:
noise_flat = np.reshape(noise, (-1, nc))
cov = np.dot(np.conj(noise_flat).T, noise_flat)


noise_white = bart(1, 'whiten', noise_flat[:,None,None,:], noise_flat[:,None,None,:]).reshape((-1, nc))
ksp_white = bart(1, 'whiten', ksp[:,:,None,:], noise_flat[:,None,None,:]).squeeze()
sens_white = bart(1, 'whiten', sens[:,:,None,:], noise_flat[:,None,None,:]).squeeze()


cov_white = np.dot(np.conj(noise_white).T, noise_white)

plt.figure(figsize=(16,4))
plt.subplot(1,2,1)
plt.imshow(abs(cov.squeeze()))
plt.title('Covariance matrix before pre-whitening')
plt.subplot(1, 2, 2)
plt.imshow(abs(cov_white.squeeze()))
plt.title('Covariance matrix after pre-whitening')

cimg_white = bart(1, 'fft -iu 3', ksp_white).squeeze()

plt.figure(figsize=(16,20))
for i in range(nc):
    plt.subplot(1, nc, i+1)
    plt.imshow(abs(cimg_white[:,:,i]).squeeze(), cmap='gray')
    plt.title('Coil image {}'.format(i))
    
plt.figure(figsize=(16,20))
for i in range(nc):
    plt.subplot(1, nc, i+1)
    plt.imshow(abs(sens_white[:,:,i]).squeeze(), cmap='gray')
    plt.title('Sensitivity map {}'.format(i))

## Visualize the sampling pattern

In [ ]:
assert np.linalg.norm(mask - (np.linalg.norm(ksp, axis=2) != 0)) < 1E-5, 'mask does not match data!'
    
plt.figure(figsize=(4, 4))
plt.imshow(mask, cmap='gray')
plt.title('sampling pattern')

R = np.prod(np.shape(mask)) / np.sum(mask)
print('Acceleration factor: {}'. format(R))

## Reconstruct the data before and after noise whitening

In [ ]:
reco = bart(1, 'pics -S -i 150', ksp[:,:,None,:], sens[:,:,None,:])
reco_white = bart(1, 'pics -S -i 30', ksp_white[:,:,None,:], sens_white[:,:,None,:])


plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plt.imshow(abs(reco).squeeze(), cmap='gray')
plt.title('Reconstruction before noise whitening')

plt.subplot(1, 2, 2)
plt.imshow(abs(reco_white).squeeze(), cmap='gray')
plt.title('Reconstruction after noise whitening')


## Estimate G-factor
In parallel imaging, noise is amplified in the reconstruction in a spatially-varying way that depends on the specific geometry of the coils. This is formalized with the g-factor.

### Pseudo Multiple Replica Method
We can estimate the g-factor through a Monte Carlo approach, where we perform many reconstructions on different instances of noise. As a side-benefit, we can also scale the reconstruction to SNR units.

The setup is the following:
1. Add noise to kspace and reconstruct. Repeat N_MC times
2. Reconstruct noise without kspace and without under-sampling. Repeat N_MC times
3. g_factor = std((1)) / (std((2)) * sqrt(R))

We will use bart to perform the reconstructions


## repeat reconstruction with N_MC noise instances

In [ ]:
n_mc = 30

recons = np.zeros((nx, ny, n_mc), dtype=np.complex)

tic = time.time()
for i in range(n_mc):

    ksp_noise = np.random.randn(*ksp.shape) + 1j*np.random.randn(*ksp.shape)
    ksp_noise = mask[:,:,None] * (ksp_white + ksp_noise)
    
    recons[:,:,i] = bart(1, 'pics -S -i 100', ksp_noise[:,:,None,:], sens_white[:,:,None,:])
    
toc = time.time()

print('Done ({} s)'.format(toc - tic))

## repeat reconstruction with N_MC noise-only instances

In [ ]:
recons_noise = np.zeros((nx, ny, n_mc), dtype=np.complex)

tic = time.time()
for i in range(n_mc):
    
    ksp_noise = np.random.randn(*ksp.shape) + 1j*np.random.randn(*ksp.shape)
    recons_noise[:,:,i] = bart(1, 'pics -g -S -i 25', ksp_noise[:,:,None,:], sens_white[:,:,None,:])
    
toc = time.time()

print('Done ({} s)'.format(toc - tic))

## compute g-factor

In [ ]:
recons_std = np.std(recons.real, axis=2)
recon_noise_std = np.std(recons_noise, axis=2)

gfactor = np.divide(recons_std, recon_noise_std, where=abs(recons[:,:,0].squeeze()) != 0) / np.sqrt(R)

max_gf = np.max(gfactor)

print('Max g-factor value: {}'.format(max_gf))


plt.figure(figsize=(16, 8))
plt.imshow(gfactor, vmax=max_gf)
plt.title('Estimated g-factor')
plt.colorbar()